In [1]:
from __future__ import print_function

import os.path
import quopri
from io import BytesIO
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from apiclient import errors
import email
import base64
from IPython.display import display, HTML
from escpos.printer import Usb
import usb.core
import imgkit
from PIL import Image
import time


p = Usb(0x04b8, 0x0202)

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly', 'https://www.googleapis.com/auth/gmail.modify']

def unstar_message(service, msg_id):
    user_id = 'me'

    service.users().messages().modify(userId= user_id, id=msg_id, body= {'removeLabelIds': ["STARRED"]}).execute()

    try:
        service.users().messages().modify(userId= user_id, id=msg_id, body= {'removeLabelIds': ["STARRED"]}).execute()
        print("Pass")
    except:
        print("Error")

def get_message(service, msg_id):
    """
    Search the inbox for specific message by ID and return it back as a 
    clean string. String may contain Python escape characters for newline
    and return line. 
    
    PARAMS
        service: the google api service object already instantiated
        user_id: user id for google api service ('me' works here if
        already authenticated)
        msg_id: the unique id of the email you need
    RETURNS
        A string of encoded text containing the message body
    """

    user_id = 'me'
    service = get_service()

    try:
        # grab the message instance
        message = service.users().messages().get(userId=user_id, id=msg_id,format='raw').execute()

        # decode the raw string, ASCII works pretty well here
        msg_str = base64.urlsafe_b64decode(message['raw'].encode('ASCII'))

        # grab the string from the byte object
        mime_msg = email.message_from_bytes(msg_str)

        # check if the content is multipart (it usually is)
        content_type = mime_msg.get_content_maintype()
        if content_type == 'multipart':
            # there will usually be 2 parts the first will be the body in text
            # the second will be the text in html
            parts = mime_msg.get_payload()

            # return the encoded text
            final_content = parts[1].get_payload()
            return final_content

        elif content_type == 'text':
            return mime_msg.get_payload()

        else:
            return ""
            print("\nMessage is not text or multipart, returned an empty string")
    # unsure why the usual exception doesn't work in this case, but 
    # having a standard Exception seems to do the trick
    except Exception:
        print("An error occured: ")



def search_message(service):
    """
    Search the inbox for emails using standard gmail search parameters
    and return a list of email IDs for each result
    PARAMS:
        service: the google api service object already instantiated
        user_id: user id for google api service ('me' works here if
        already authenticated)
        search_string: search operators you can use with Gmail
        (see https://support.google.com/mail/answer/7190?hl=en for a list)
    RETURNS:
        List containing email IDs of search query
    """
    user_id = 'me'
    search_string = 'Venmo'

    try:
        # initiate the list for returning
        list_ids = []

        # get the id of all messages that are in the search string
        search_ids = service.users().messages().list(userId=user_id, q='is:starred').execute()
        
        # if there were no results, print warning and return empty string
        try:
            ids = search_ids['messages']

        except KeyError:
            print("WARNING: the search queried returned 0 results")
            print("returning an empty string")
            return ""

        if len(ids)>-1:
            for msg_id in ids:
                list_ids.append(msg_id['id'])
            return(list_ids)

        else:
            list_ids.append(ids['id'])
            return list_ids
        
    except (errors.HttpError):
        print("An error occured: ")


def get_service():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('gmail', 'v1', credentials=creds)

    return service




def print_receipts(service):
    messages = search_message(service)
    print(messages)

    for i in messages:

        message = get_message(service, i)
        chart = HTML(message)

        message = bytes(message, 'ascii')

        OutputFile = BytesIO()
        InputFile = BytesIO(message)
        quopri.decode(InputFile, OutputFile)
        output = OutputFile.getvalue()



        Func = open("GFG-1.html","w", encoding="utf-8")
        Func.write(output.decode('utf-8'))
        Func.close()
        imgkit.from_file("GFG-1.html", 'out.jpg')
        image = Image.open("out.jpg")
        w, h = image.size
        image = image.crop((0, 0, w, h))
        image = image.resize((w + 350, h + 450))
        image.save("out.jpg")
        display(image)
        p.image("out.jpg")
        p.cut()
        print("File Printed")
        unstar_message(service, i)
              

p.cut()


In [2]:
service = get_service()

while True == True:
    print_receipts(service)
    time.sleep(3)
    

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=487335719322-tbp1q2afs7ra0gj514graalkt35qko16.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58588%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=QFujSy8Icb8kR9xpz09jRaIk6AYCQ7&access_type=offline
['184f02b753d84e40', '184dfbb5c771d640', '184dac5bbae2c04d', '184d6037617e950d', '184d4b7529dd3e1e', '184d4b71e16c86bb', '184d484a68b2cf6d', '184d47569cf72b63', '184d4731803b8ed9', '184bc792eb0b9bd5', '184bbf8f2a025821', '184bb0d522e76239', '184bb0cfbbc2ab21', '184bafd8ef43f16e', '184b7643106eace7', '184b6035a138fe04', '184b1f77daf11136', '184aa899e1c3b680', '184aa5eb6e2a76d4', '184a69f879856084', '184a60078e53054f', '184a512bd429984f', '184a2f7ef621716a', '184a29b0c00d7ff1', '184a2996e1167cff', '184a2978cacae522', '184a2930fd95f5a0', '184a29004a55633b', '184a28fca503bfc0', 

NameError: name 'p' is not defined

In [1]:
from __future__ import print_function

import os.path
import quopri
from io import BytesIO
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from apiclient import errors
import email
import base64
from IPython.display import display, HTML
from escpos.printer import Usb
import escpos.image as img
import usb.core
import imgkit
from PIL import Image
import time

p = Usb(0x04b8, 0x0202)

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly', 'https://www.googleapis.com/auth/gmail.modify']



def unstar_message(service, msg_id):
    user_id = 'me'

    service.users().messages().modify(userId= user_id, id=msg_id, body= {'removeLabelIds': ["STARRED"]}).execute()

    try:
        service.users().messages().modify(userId= user_id, id=msg_id, body= {'removeLabelIds': ["STARRED"]}).execute()
        print("Pass")
    except:
        print("Error")

def get_message(service, msg_id):
    """
    Search the inbox for specific message by ID and return it back as a 
    clean string. String may contain Python escape characters for newline
    and return line. 
    
    PARAMS
        service: the google api service object already instantiated
        user_id: user id for google api service ('me' works here if
        already authenticated)
        msg_id: the unique id of the email you need
    RETURNS
        A string of encoded text containing the message body
    """

    user_id = 'me'
    #service = get_service()

    try:
        # grab the message instance
        message = service.users().messages().get(userId=user_id, id=msg_id,format='raw').execute()

        # decode the raw string, ASCII works pretty well here
        msg_str = base64.urlsafe_b64decode(message['raw'].encode('ASCII'))

        # grab the string from the byte object
        mime_msg = email.message_from_bytes(msg_str)

        # check if the content is multipart (it usually is)
        content_type = mime_msg.get_content_maintype()
        if content_type == 'multipart':
            # there will usually be 2 parts the first will be the body in text
            # the second will be the text in html
            parts = mime_msg.get_payload()

            # return the encoded text
            final_content = parts[1].get_payload()
            return final_content

        elif content_type == 'text':
            return mime_msg.get_payload()

        else:
            return ""
            print("\nMessage is not text or multipart, returned an empty string")
    # unsure why the usual exception doesn't work in this case, but 
    # having a standard Exception seems to do the trick
    except Exception:
        print("An error occured: ")



def search_message(service):
    """
    Search the inbox for emails using standard gmail search parameters
    and return a list of email IDs for each result
    PARAMS:
        service: the google api service object already instantiated
        user_id: user id for google api service ('me' works here if
        already authenticated)
        search_string: search operators you can use with Gmail
        (see https://support.google.com/mail/answer/7190?hl=en for a list)
    RETURNS:
        List containing email IDs of search query
    """
    user_id = 'me'
    search_string = 'Venmo'

    try:
        # initiate the list for returning
        list_ids = []

        # get the id of all messages that are in the search string
        search_ids = service.users().messages().list(userId=user_id, q='is:starred').execute()
        
        # if there were no results, print warning and return empty string
        try:
            ids = search_ids['messages']

        except KeyError:
            print("WARNING: the search queried returned 0 results")
            print("returning an empty string")
            return ""

        if len(ids)>-1:
            for msg_id in ids:
                list_ids.append(msg_id['id'])
            return(list_ids)

        else:
            list_ids.append(ids['id'])
            return list_ids
        
    except (errors.HttpError):
        print("An error occured: ")


def get_service():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('gmail', 'v1', credentials=creds)

    return service




def print_receipts(service):
    messages = search_message(service)
    print(messages)

    for i in messages:

        message = get_message(service, i)
        chart = HTML(message)

        #print(message)

        message = bytes(message, 'ascii')

        OutputFile = BytesIO()
        InputFile = BytesIO(message)
        quopri.decode(InputFile, OutputFile)
        output = OutputFile.getvalue()

        #print(output.decode('utf-8'))

        #display(chart)


        Func = open("GFG-1.html","w",encoding="utf-8")
        Func.write(output.decode('utf-8'))
        Func.close()

        path = r'C:\Program Files (x86)\wkhtmltopdf\bin\wkhtmltoimage.exe'
        config = imgkit.config(wkhtmltoimage=path)
        imgkit.from_file("GFG-1.html", 'out.jpg', config=config)


        '''
        path_wkhtmltopdf = r'C:\Program Files (x86)\wkhtmltopdf\bin\wkhtmltopdf.exe'
        config = imgkit.configuration(wkhtmltopdf=path_wkhtmltopdf)
        imgkit.from_file("GFG-1.html", "out.pdf", configuration=config)
        '''

        image = Image.open("out.jpg")
        w, h = image.size
        image = image.crop((0, 0, w, h))
        image = image.resize((w + 350, h + 450))
        image.save("out.jpg")
        #w = img.EscposImage("out.jpg").width
        #img.EscposImage("out.jpg").center(max_width=w)
        #img.EscposImage("out.jpg").to_raster_format()
        p.image("out.jpg")
        p.cut()
        print("File Printed")
        unstar_message(service, i)
              

#p.cut()


service = get_service()

while True == True:
    print_receipts(service)
    time.sleep(3)

RefreshError: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})